# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import seaborn as sns
from scipy.stats import linregress
from scipy import stats
import statsmodels.api as sm
import time

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath= "my_Weather_data.csv"

In [3]:
# Read with Pandas
df = pd.read_csv(filepath)
df.head()

,latitude,longitude,country,cloudiness,timestamp,cify_name,wind,temp,max_temp,pressure,humidity
0,5.4112,100.3354,MY,40.0,2022-02-04 20:22:32,George Town,3.44,77.95,79.90,1008.0,93.0
1,-41.8697,-73.8203,CL,80.0,2022-02-04 20:26:05,Ancud,9.89,69.22,69.22,1021.0,69.0
2,68.5333,146.4167,RU,52.0,2022-02-04 20:26:05,Belaya Gora,1.07,-39.33,-39.33,1027.0,100.0
3,56.9064,118.2631,RU,99.0,2022-02-04 20:26:05,Chara,1.72,-15.00,-15.00,1038.0,86.0
4,-0.6000,73.0833,MV,60.0,2022-02-04 20:26:06,Hithadhoo,10.38,81.79,81.79,1009.0,73.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507 entries, 0 to 506
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   latitude    507 non-null    float64
 1   longitude   507 non-null    float64
 2   country     503 non-null    object 
 3   cloudiness  507 non-null    float64
 4   timestamp   507 non-null    object 
 5   cify_name   507 non-null    object 
 6   wind        507 non-null    float64
 7   temp        507 non-null    float64
 8   max_temp    507 non-null    float64
 9   pressure    507 non-null    float64
 10  humidity    507 non-null    float64
dtypes: float64(8), object(3)
memory usage: 43.7+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#create heatmap
locations = df.loc[:, ["latitude", "longitude"]]

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df['humidity'],
    max_intensity=20,point_radius = 5
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#loacte ideal weather cities
mask = (df.temp <80) & (df.temp >70) & (df.wind<10) & (df.cloudiness == 0)

g_cities = df.loc[mask]

g_cities

,latitude,longitude,country,cloudiness,timestamp,cify_name,wind,temp,max_temp,pressure,humidity
11,10.3744,10.7288,NG,0.0,2022-02-04 20:26:08,Bara,8.46,71.87,71.87,1015.0,17.0
30,-28.7667,114.6000,AU,0.0,2022-02-04 20:26:24,Geraldton,5.75,70.21,70.21,1006.0,43.0
41,-29.6641,22.7474,ZA,0.0,2022-02-04 20:26:28,Prieska,7.07,79.97,79.97,1014.0,25.0
48,22.8909,-109.9124,MX,0.0,2022-02-04 20:21:48,Cabo San Lucas,3.00,72.84,75.51,1016.0,50.0
66,14.8961,-24.4956,CV,0.0,2022-02-04 20:27:15,São Filipe,9.86,73.81,73.81,1014.0,72.0
70,22.0752,-159.3190,US,0.0,2022-02-04 20:27:16,Kapaa,1.01,77.52,78.48,1019.0,70.0
124,20.8947,-156.4700,US,0.0,2022-02-04 20:25:24,Kahului,5.75,79.30,83.46,1019.0,60.0
225,-23.5333,148.1667,AU,0.0,2022-02-04 20:29:23,Emerald,8.05,71.74,71.74,1008.0,73.0
328,21.3156,-158.0072,US,0.0,2022-02-04 20:31:51,‘Ewa Beach,0.00,76.80,79.09,1019.0,61.0
471,8.9333,8.6000,NG,0.0,2022-02-04 20:35:09,Wamba,4.50,73.62,73.62,1013.0,17.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# geocoordinates
target_coordinates = f"{5.4112}, {100.3354}"
target_search = "hotel"

target_radius = 50000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

places_data = response.json()

print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])
print(places_data["results"][0]["rating"])

Nam Keng Hotel
17, Lebuh Cintra, George Town
4.6


In [8]:
#create empty lists
hotels = []
ratings = []
addresses = []
cities = []
countries = []
lats = []
lngs = []

#loop through ideal cities
for index_, row in g_cities.iterrows():
    print(row.cify_name)
    target_coordinates = f"{row.latitude}, {row.longitude}"
    target_search = "hotel"    
    target_radius = 50000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    try:
        places_data = response.json()
        hotel = places_data["results"][0]["name"]
        address = places_data["results"][0]["vicinity"]
        rating = places_data["results"][0]["rating"]
    
        hotels.append(hotel)
        addresses.append(address)
        ratings.append(rating)
        cities.append(row.cify_name)
        countries.append(row.country)
        lats.append(row.latitude)
        lngs.append(row.longitude)
        
    except Exception as e:
        print(e)
        hotels.append(np.nan)
        addresses.append(np.nan)
        ratings.append(np.nan)
        cities.append(np.nan)
        countries.append(np.nan)
        lats.append(np.nan)
        lngs.append(np.nan)
        
    time.sleep(1)


Bara
Geraldton
Prieska
Cabo San Lucas
São Filipe
Kapaa
Kahului
Emerald
‘Ewa Beach
Wamba
Nouakchott


In [12]:
#create new dataframe
hotels_df = pd.DataFrame()

hotels_df["City"] = cities
hotels_df["Country"] = countries
hotels_df["Hotel"] = hotels
hotels_df["Rating"] = ratings
hotels_df["Address"] = addresses
hotels_df["Rating"] = ratings
hotels_df["Latitude"] = lats
hotels_df["Longitude"] = lngs

hotels_df

,City,Country,Hotel,Rating,Address,Latitude,Longitude
0,Bara,NG,Custodian Hotel Gombe,3.8,Gombe,10.3744,10.7288
1,Geraldton,AU,Northampton Motor Hotel,4.1,"220 Hampton Rd, Northampton",-28.7667,114.6000
2,Prieska,ZA,Riverview Lodge Prieska,4.2,"Lang St, Prieska",-29.6641,22.7474
3,Cabo San Lucas,MX,Hotel El Ganzo,4.5,"Blvd. Tiburon s/n, La Playita, San José del Cabo",22.8909,-109.9124
4,São Filipe,CV,Hotel Xaguate,3.5,"Sao Filipe, Rua da Posto, Fogo",14.8961,-24.4956
5,Kapaa,US,Kauai Beach Resort & Spa,4.3,"4331 Kauai Beach Dr, Lihue",22.0752,-159.3190
6,Kahului,US,Hotel Lanai,4.6,"828 Lanai Ave, Lanai City",20.8947,-156.4700
7,Emerald,AU,Emerald Central Hotel,3.5,"13 Opal St, Emerald",-23.5333,148.1667
8,‘Ewa Beach,US,Waikiki Resort Hotel,4.0,"2460 Koa Ave, Honolulu",21.3156,-158.0072
9,Wamba,NG,Nassarawa Luxury Hotel,4.3,"Lafia - Shandam Rd, Lafia",8.9333,8.6000


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations2 = hotels_df[["Latitude", "Longitude"]]

In [21]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations2,info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)

fig


# Display figure


Figure(layout=FigureLayout(height='420px'))